In [1]:
using Revise

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /home/andy/.julia/packages/Revise/C272c/src/Revise.jl:1219


In [2]:
using Pkg; Pkg.activate("../")
using julia2wat
#include("../src/julia2wat.jl")

Activating environment at `~/dev/julia2wat/Project.toml`


┌ Info: Precompiling julia2wat [683821a8-2d1a-4f23-b146-c2dd114290c7]
└ @ Base loading.jl:1273
  ** incremental compilation may be fatally broken for this module **



In [31]:
function f(N)
    s=0.0
    i=0
    while i<100
        i+=1
        s+=N
    end
    return s
end

kaka(a,b,c,d) = (a+b)*(c+d)

kaka (generic function with 1 method)

In [24]:
name = "../wasm/kek"

#@code_wat kaka(1.0, 2.0, 3.0, 4.0) :($name)
@code_wat f(3.0) :($name)

Array{Any}((9,))
  1: Nothing nothing
  2: Core.PhiNode
    edges: Array{Any}((2,))
      1: Int64 1
      2: Int64 8
    values: Array{Any}((2,))
      1: Float64 0.0
      2: Core.SSAValue
        id: Int64 7
  3: Core.PhiNode
    edges: Array{Any}((2,))
      1: Int64 1
      2: Int64 8
    values: Array{Any}((2,))
      1: Int64 0
      2: Core.SSAValue
        id: Int64 6
  4: Expr
    head: Symbol call
    args: Array{Any}((3,))
      1: GlobalRef
        mod: Module Base
        name: Symbol slt_int
      2: Core.SSAValue
        id: Int64 3
      3: Int64 10
  5: Expr
    head: Symbol gotoifnot
    args: Array{Any}((2,))
      1: Core.SSAValue
        id: Int64 4
      2: Int64 9
  6: Expr
    head: Symbol call
    args: Array{Any}((3,))
      1: GlobalRef
        mod: Module Base
        name: Symbol add_int
      2: Core.SSAValue
        id: Int64 3
      3: Int64 1
  7: Expr
    head: Symbol call
    args: Array{Any}((3,))
      1: GlobalRef
        mod: Module Base
        

In [27]:
function f2(N)
    s=0.0
    for i=1:100
        s+=N
    end
    return s
end

f2 (generic function with 1 method)

In [32]:
@code_wat f2(3.0) :($name)

Array{Any}((16,))
  1: Expr
    head: Symbol gotoifnot
    args: Array{Any}((2,))
      1: Bool true
      2: Int64 15
  2: Core.PhiNode
    edges: Array{Any}((2,))
      1: Int64 1
      2: Int64 14
    values: Array{Any}((2,))
      1: Float64 0.0
      2: Core.SSAValue
        id: Int64 4
  3: Core.PhiNode
    edges: Array{Any}((2,))
      1: Int64 1
      2: Int64 14
    values: Array{Any}((2,))
      1: Int64 1
      2: Core.SSAValue
        id: Int64 10
  4: Expr
    head: Symbol call
    args: Array{Any}((3,))
      1: GlobalRef
        mod: Module Base
        name: Symbol add_float
      2: Core.SSAValue
        id: Int64 2
      3: Core.SlotNumber
        id: Int64 2
  5: Expr
    head: Symbol call
    args: Array{Any}((3,))
      1: GlobalRef
        mod: Module Base
        name: Symbol ===
      2: Core.SSAValue
        id: Int64 3
      3: Int64 100
  ...
  12: Expr
    head: Symbol call
    args: Array{Any}((2,))
      1: GlobalRef
        mod: Module Base
        name: 

KeyError: KeyError: key 11 not found

In [29]:
#cinfo=@code_typed kaka(1.0, 2.0, 3.0, 4.0)
cinfo=@code_typed f2(3.0)
cinfo=cinfo[1]
@code_llvm debuginfo=:none f2(3.0)


define double @julia_f2_17806(double) {
top:
  br label %L2

L2:                                               ; preds = %L2, %top
  %value_phi = phi double [ 0.000000e+00, %top ], [ %1, %L2 ]
  %value_phi1 = phi i64 [ 1, %top ], [ %3, %L2 ]
  %1 = fadd double %value_phi, %0
  %2 = icmp eq i64 %value_phi1, 100
  %3 = add nuw nsw i64 %value_phi1, 1
  br i1 %2, label %L15, label %L2

L15:                                              ; preds = %L2
  ret double %1
}


In [32]:
cinfo = @code_typed debuginfo=:none f(3.0)
cinfo=cinfo[1]
@code_llvm debuginfo=:none f(3.0)


define double @julia_f_17808(double) {
top:
  br label %L6

L6:                                               ; preds = %top, %L6
  %value_phi13 = phi i64 [ 0, %top ], [ %1, %L6 ]
  %value_phi2 = phi double [ 0.000000e+00, %top ], [ %2, %L6 ]
  %1 = add nuw nsw i64 %value_phi13, 1
  %2 = fadd double %value_phi2, %0
  %exitcond = icmp eq i64 %1, 100
  br i1 %exitcond, label %L9, label %L6

L9:                                               ; preds = %L6
  ret double %2
}


In [25]:
cinfo.slotnames

5-element Array{Symbol,1}:
 Symbol("#self#")
 :N              
 :s              
 Symbol("")      
 :i              

In [ ]:
cinfo.code[2]

In [ ]:
cinfo.ssavaluetypes

In [ ]:
k(x) = 3.1*exp(x)

In [ ]:
cinfo=@code_typed k(3.0)
cinfo=cinfo[1]

In [ ]:
cinfo.code

In [ ]:
cinfo.ssavaluetypes

In [ ]:
#cinfo.ssaflags

In [ ]:
?Core.CodeInfo

In [ ]:
@code_lowered f(2.0, 3)

In [ ]:
?@code_typed